In [3]:
import re
import pandas as pd
import numpy as np

## 1. Create is_vowel func:

In [23]:
def is_vowel(txt):
    regexp = r'^[aeiouAEIOU]$'
    try:
        if re.search(regexp, txt)[0] in ['a', 'e', 'i', 'o', 'u']:
            return True
    except TypeError:
        return False

In [28]:
is_vowel(txt = 'a'), is_vowel(txt = 'z')

(True, False)

## 2. is_validuser:

In [32]:
def is_validuser(usr):
    regexp = r'^[a-z]\w+_?'
    if len(usr) > 32:
        return False
    else:
        return bool(re.search(regexp,usr))

In [33]:
is_validuser('a'*33), is_validuser('test_name')

(False, True)

## 3. Phone number regex:
- (210) 867 5309
- +1 210.867.5309
- 867-5309
- 210-867-5309

In [35]:
df = pd.DataFrame()
df['phone'] = [
    '(210) 867 5309',
    '+1 210.867.5309',
    '867-5309',
    '210-867-5309'
]
df

,phone
0,(210) 867 5309
1,+1 210.867.5309
2,867-5309
3,210-867-5309


In [36]:
phone_regex = re.compile(
'''
^
(?P<country_code>\+\d+)?
\D*?
(?P<area_code>\d{3})?
\D*?
(?P<exchange_code>\d{3})
\D*?
(?P<line_number>\d{4})
\D*
$
''', re.VERBOSE)

In [37]:
number_split = df.phone.str.extract(phone_regex)
number_split

,country_code,area_code,exchange_code,line_number
0,NaN,210,867,5309
1,+1,210,867,5309
2,NaN,NaN,867,5309
3,NaN,210,867,5309


## 4. Dates Regex:
- 02/04/19
- 02/05/19
- 02/06/19
- 02/07/19
- 02/08/19
- 02/09/19
- 02/10/19

In [40]:
dates = pd.Series([
'02/04/19',
'02/05/19',
'02/06/19',
'02/07/19',
'02/08/19',
'02/09/19',
'02/10/19']
)

In [42]:
regexp = r'(\d{2})/(\d{2})/(\d{2})'
dates = dates.str.replace(regexp, r'\3-\1-\2')

<ipython-input-42-3e68cd4ebf3e>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dates = dates.str.replace(regexp, r'\3-\1-\2')


In [43]:
dates

0    19-02-04
1    19-02-05
2    19-02-06
3    19-02-07
4    19-02-08
5    19-02-09
6    19-02-10
dtype: object

## Log Regex:
> GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58
POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58
GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58

In [44]:
log = """
GET /api/v1/sales?page=86 [16/Apr/2019:193452+0000] HTTP/1.1 {200} 510348 "python-requests/2.21.0" 97.105.19.58
POST /users_accounts/file-upload [16/Apr/2019:193452+0000] HTTP/1.1 {201} 42 "User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36" 97.105.19.58
GET /api/v1/items?page=3 [16/Apr/2019:193453+0000] HTTP/1.1 {429} 3561 "python-requests/2.21.0" 97.105.19.58
"""

In [45]:
regexp = r'''
^
(?P<method>GET|POST)
\s
(?P<path>/[/\w\-\?=]+)
\s
\[(?P<timestamp>.+)\]
\s
(?P<http_version>HTTP/\d+\.\d+)
\s
\{(?P<status_code>\d+)\}
\s
(?P<bytes>\d+)
\s
"(?P<user_agent>.+)"
\s
(?P<ip>\d+\.\d+\.\d+\.\d+)
$
'''

In [47]:
[re.search(regexp, l, re.VERBOSE).groupdict() for l in log.strip().split('\n')]

[{'method': 'GET',
  'path': '/api/v1/sales?page=86',
  'timestamp': '16/Apr/2019:193452+0000',
  'http_version': 'HTTP/1.1',
  'status_code': '200',
  'bytes': '510348',
  'user_agent': 'python-requests/2.21.0',
  'ip': '97.105.19.58'},
 {'method': 'POST',
  'path': '/users_accounts/file-upload',
  'timestamp': '16/Apr/2019:193452+0000',
  'http_version': 'HTTP/1.1',
  'status_code': '201',
  'bytes': '42',
  'user_agent': 'User-Agent: Mozilla/5.0 (X11; Fedora; Fedora; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.86 Safari/537.36',
  'ip': '97.105.19.58'},
 {'method': 'GET',
  'path': '/api/v1/items?page=3',
  'timestamp': '16/Apr/2019:193453+0000',
  'http_version': 'HTTP/1.1',
  'status_code': '429',
  'bytes': '3561',
  'user_agent': 'python-requests/2.21.0',
  'ip': '97.105.19.58'}]